In [6]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt


In [7]:
import numpy as np
import pandas as pd
import datetime as dt
import json
from flask import Flask, jsonify


In [8]:
## Reflect Tables into SQLAlchemy ORM

In [9]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [10]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
inspector = inspect(engine)

In [11]:
# reflect an existing database into a new model
Base = automap_base()


In [12]:
# reflect the tables
Base.prepare(engine, reflect=True)


In [13]:
# View all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [14]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [15]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [16]:
# Find the most recent date in the data set.
inspector.get_table_names()

['measurement', 'station']

In [17]:
columns = inspector.get_columns('measurement')
for c in columns:
    print (c['name'], c['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [18]:
# Find the last date

latest_date= session.query(Measurement.date).order_by(Measurement.date.desc()).first()
latest_date

('2017-08-23')

In [19]:
#Calcaulate the date one year from the last daye in data set:
one_year_date = dt.date(2017, 8, 23)-dt.timedelta(days=365)
print(one_year_date)

2016-08-23


In [20]:
# Perform a query to retrieve the data and precipitation scores
result = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_date).all()